# Arrest Prediction — v1 (HistGradientBoosting + Engineered Features)
**Goal:** Beat RF v0 (PR-AUC ≈ 0.623) by adding time features, rare bucketing, and frequency encodings, then training a HistGradientBoosting baseline.

**Dataset:** data/processed/arrest_features.csv  
**Target:** arrest (0/1)  
**Artifacts:** saved to notebooks/artifacts/

In [33]:
# Core imports 
import os, time, json, numpy as np, pandas as pd 
from pathlib import Path 
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"


# Modeling + metrics 
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    average_precision_score, roc_auc_score, classification_report, 
    confusion_matrix, precision_recall_curve, roc_curve
)
from sklearn.experimental import enable_hist_gradient_boosting 
from sklearn.ensemble import HistGradientBoostingClassifier

import matplotlib.pyplot as plt
from scipy.stats import loguniform, randint
import tempfile
from sklearn.metrics import precision_recall_fscore_support

# Paths
REPO = Path.cwd()
while REPO.name != "chicago-crime-pipeline" and REPO.parent != REPO:
    REPO = REPO.parent
DATA = REPO / "data" / "processed"
ART = REPO / "notebooks" / "artifacts"
ART.mkdir(parents=True, exist_ok=True)

# Load
df = pd.read_csv(DATA / "arrest_features.csv")
assert "arrest" in df.columns
print(df.shape, df["arrest"].value_counts(dropna=False).to_dict())

# Split (same seed/stratify as v0)
TARGET = "arrest"
y = df[TARGET].astype(int).values
X = df.drop(columns=[TARGET]).copy()
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
X_train.shape, X_test.shape




(10482, 10) {0: 8972, 1: 1510}


((8385, 9), (2097, 9))

In [21]:
def slice_metrics(X_df, y_true, proba, threshold, slice_col, min_support=40):
    """
    Compute precision/recall/F1 per value of a categorical slice column.
    Saves nothing; just returns a DataFrame. You can write it to CSV after.
    """
    if slice_col not in X_df.columns:
        print(f"[skip] slice column not found: {slice_col}")
        return None

    df = pd.DataFrame({
        slice_col: X_df[slice_col],
        "y": y_true,
        "pred": (proba >= threshold).astype(int)
    })

    rows = []
    for val, g in df.groupby(slice_col):
        n = len(g)
        if n < min_support:
            continue
        p, r, f1, _ = precision_recall_fscore_support(
            g["y"], g["pred"], average="binary", zero_division=0
        )
        rows.append({
            slice_col: val, "support": int(n),
            "precision": float(p), "recall": float(r), "f1": float(f1)
        })

    if not rows:
        print(f"[note] no slices with support ≥ {min_support} for {slice_col}")
        return None

    return pd.DataFrame(rows).sort_values("f1", ascending=False).reset_index(drop=True)

In [ ]:
# [31] — REPLACE ENTIRE CELL
# Rebuild engineered features from X_train/X_test
X_train_fe = X_train.copy()
X_test_fe  = X_test.copy()

# Weekday from date
for Xdf in (X_train_fe, X_test_fe):
    Xdf["weekday"] = pd.to_datetime(Xdf["date"]).dt.day_name()

# Hour bins (object dtype for OHE)
bins   = [0,6,12,18,24]
labels = ["00-05","06-11","12-17","18-23"]
for Xdf in (X_train_fe, X_test_fe):
    Xdf["hour_bin"] = pd.cut(Xdf["hour"].astype(int), bins=bins, right=False, labels=labels).astype(object)

# Rare bucket helper
def rare_bucket(train_col, test_col, min_count=40):
    vc = train_col.value_counts()
    keep = set(vc[vc >= min_count].index)
    return (train_col.where(train_col.isin(keep), "__RARE__"),
            test_col.where(test_col.isin(keep), "__RARE__"))

# Rare-bucket high-card categoricals if present
for col in ["location_description", "primary_type"]:
    if col in X_train_fe.columns:
        X_train_fe[col], X_test_fe[col] = rare_bucket(X_train_fe[col], X_test_fe[col], 40)

# Frequency encodings
def add_freq_encode(col):
    freq = X_train_fe[col].astype(object).value_counts(normalize=True)
    X_train_fe[f"{col}_freq"] = X_train_fe[col].map(freq).astype("float64").fillna(0.0).to_numpy()
    X_test_fe[f"{col}_freq"]  = X_test_fe[col].map(freq).astype("float64").fillna(0.0).to_numpy()

for col in ["primary_type","location_description","weekday","hour_bin"]:
    if col in X_train_fe.columns:
        add_freq_encode(col)

# Target mean encoding (primary_type → arrest rate)
if "primary_type" in X_train_fe.columns:
    arrest_rate = pd.Series(y_train).groupby(X_train_fe["primary_type"]).mean()
    X_train_fe["ptype_arrest_rate"] = X_train_fe["primary_type"].map(arrest_rate)
    X_test_fe["ptype_arrest_rate"]  = X_test_fe["primary_type"].map(arrest_rate).fillna(float(arrest_rate.mean()))
else:
    X_train_fe["ptype_arrest_rate"] = 0.0
    X_test_fe["ptype_arrest_rate"]  = 0.0

# Interaction: primary_type × hour_bin (then rare-bucket)
if set(["primary_type","hour_bin"]).issubset(X_train_fe.columns):
    X_train_fe["ptype_x_hourbin"] = X_train_fe["primary_type"].astype(str) + "_" + X_train_fe["hour_bin"].astype(str)
    X_test_fe["ptype_x_hourbin"]  = X_test_fe["primary_type"].astype(str)  + "_" + X_test_fe["hour_bin"].astype(str)
    X_train_fe["ptype_x_hourbin"], X_test_fe["ptype_x_hourbin"] = rare_bucket(
        X_train_fe["ptype_x_hourbin"], X_test_fe["ptype_x_hourbin"], min_count=30
    )
else:
    X_train_fe["ptype_x_hourbin"] = "__MISSING__"
    X_test_fe["ptype_x_hourbin"]  = "__MISSING__"

In [ ]:
 
# Desired lists
cat_cols_fe = ["date","primary_type","location_description","location_grouped","weekday","hour_bin","ptype_x_hourbin"]
num_cols_fe = ["id","year","month","dow","hour","primary_type_freq","location_description_freq","weekday_freq","hour_bin_freq","ptype_arrest_rate"]

# Keep only columns that actually exist
present = set(X_train_fe.columns)
cat_cols_used = [c for c in cat_cols_fe if c in present]
num_cols_used = [c for c in num_cols_fe if c in present]

print("Using categorical:", cat_cols_used)
print("Using numeric    :", num_cols_used)

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

pre_fe = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols_used),
        ("num", "passthrough", num_cols_used),
    ],
    remainder="drop",
    verbose_feature_names_out=False,
)

Using categorical: ['date', 'primary_type', 'location_description', 'location_grouped', 'weekday', 'hour_bin', 'ptype_x_hourbin']
Using numeric    : ['id', 'year', 'month', 'dow', 'hour', 'primary_type_freq', 'location_description_freq', 'weekday_freq', 'hour_bin_freq', 'ptype_arrest_rate']


In [34]:
import numpy as np, tempfile
from sklearn.pipeline import Pipeline
from sklearn.ensemble import HistGradientBoostingClassifier
from scipy.stats import loguniform, randint

# Class imbalance → sample weights (pos gets higher weight)
pos_weight = (len(y_train) - y_train.sum()) / y_train.sum()
sw_train = np.where(y_train==1, pos_weight, 1.0)

hgb_pipe = Pipeline(steps=[
    ("pre", pre_fe),
    ("clf", HistGradientBoostingClassifier(
        random_state=42, max_bins=255,
        early_stopping=True, validation_fraction=0.1, n_iter_no_change=10
    ))
], memory=tempfile.mkdtemp())

param_dist = {
    "clf__learning_rate": loguniform(0.03, 0.2),
    "clf__max_depth": randint(3, 8),
    "clf__max_leaf_nodes": randint(24, 64),
    "clf__min_samples_leaf": randint(60, 240),
    "clf__l2_regularization": loguniform(1e-4, 0.3),
    "clf__max_iter": randint(120, 240),
}

In [35]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold

# Subsample ~5k rows for faster search
SUB_N = 5000
if len(y_train) > SUB_N:
    X_sub, _, y_sub, _ = train_test_split(
        X_train_fe, y_train, train_size=SUB_N, stratify=y_train, random_state=42
    )
    sw_sub = sw_train[:len(y_sub)]
else:
    X_sub, y_sub = X_train_fe, y_train
    sw_sub = sw_train

cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)

hgb_search = RandomizedSearchCV(
    hgb_pipe, param_distributions=param_dist,
    n_iter=6, scoring="average_precision",
    refit=True, cv=cv, n_jobs=-1, random_state=42, verbose=2
)

hgb_search.fit(X_sub, y_sub, clf__sample_weight=sw_sub)
print("Best HGB params:", hgb_search.best_params_)
print("Best CV PR-AUC:", round(hgb_search.best_score_, 4))

Fitting 2 folds for each of 6 candidates, totalling 12 fits
[CV] END clf__l2_regularization=0.00010062545641808922, clf__learning_rate=0.1970666034205786, clf__max_depth=3, clf__max_iter=195, clf__max_leaf_nodes=45, clf__min_samples_leaf=148; total time=   2.3s
[CV] END clf__l2_regularization=0.0020059560245279666, clf__learning_rate=0.18214744423753768, clf__max_depth=5, clf__max_iter=191, clf__max_leaf_nodes=44, clf__min_samples_leaf=162; total time=   2.5s
[CV] END clf__l2_regularization=0.00010062545641808922, clf__learning_rate=0.1970666034205786, clf__max_depth=3, clf__max_iter=195, clf__max_leaf_nodes=45, clf__min_samples_leaf=148; total time=   3.1s
[CV] END clf__l2_regularization=0.0035498870995898887, clf__learning_rate=0.03626531563860245, clf__max_depth=5, clf__max_iter=207, clf__max_leaf_nodes=59, clf__min_samples_leaf=163; total time=   4.5s
[CV] END clf__l2_regularization=0.001029530064265006, clf__learning_rate=0.0957705988053993, clf__max_depth=4, clf__max_iter=211, cl

In [13]:
hgb_final = hgb_search.best_estimator_
hgb_final.fit(X_train_fe, y_train, clf__sample_weight=sw_train)

,steps,"[('pre', ...), ('clf', ...)]"
,transform_input,None
,memory,'/var/folders/6z/l9wv3c...q8m0000gn/T/tmptrqagad5'
,verbose,False
,transformers,"[('cat', ...), ('num', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


In [14]:
proba_hgb = hgb_final.predict_proba(X_test_fe)[:,1]

print("HGB TEST PR-AUC:", round(average_precision_score(y_test, proba_hgb), 4))
print("HGB TEST ROC-AUC:", round(roc_auc_score(y_test, proba_hgb), 4))

# Threshold tuning
prec, rec, thr = precision_recall_curve(y_test, proba_hgb)
f1s = 2*prec*rec/(prec+rec+1e-12)
best_idx = np.nanargmax(f1s)
thr_hgb = thr[best_idx] if best_idx < len(thr) else 0.5
pred_hgb = (proba_hgb >= thr_hgb).astype(int)

print("Best threshold:", float(thr_hgb), "Best F1:", float(f1s[best_idx]))
print(classification_report(y_test, pred_hgb, digits=3))
print("Confusion:\n", confusion_matrix(y_test, pred_hgb))

HGB TEST PR-AUC: 0.6569
HGB TEST ROC-AUC: 0.8878
Best threshold: 0.6953415078096913 Best F1: 0.6265060240958864
              precision    recall  f1-score   support

           0      0.934     0.946     0.940      1795
           1      0.652     0.603     0.627       302

    accuracy                          0.897      2097
   macro avg      0.793     0.774     0.783      2097
weighted avg      0.893     0.897     0.895      2097

Confusion:
 [[1698   97]
 [ 120  182]]


In [ ]:
from pathlib import Path
if 'ART' not in globals():
    ART = Path("notebooks/artifacts"); ART.mkdir(parents=True, exist_ok=True)
stamp = time.strftime("%Y%m%d-%H%M%S")  # correct format

cols_to_check = ["weekday", "hour_bin", "primary_type"]
slice_tables = {}
for col in cols_to_check:
    tbl = slice_metrics(X_test_fe, y_test, proba_hgb, thr_hgb, col, min_support=40)
    if tbl is not None:
        slice_tables[col] = tbl
        out_path = ART / f"slice_metrics_{col}_hgb_v1_{stamp}.csv"
        tbl.to_csv(out_path, index=False)
        print(f"Saved slice metrics for {col} → {out_path}")

# quick peek
for col, tbl in slice_tables.items():
    print(f"\n=== {col}: top 5 by F1 ==="); display(tbl.head(5))
    print(f"=== {col}: bottom 5 by F1 (support ≥ 40) ==="); display(tbl.tail(5))

In [ ]:
stamp = time.strftime("%Y%m%d-%H%M%S")
metrics = {
    "timestamp": stamp,
    "model": "HGB + FE v1",
    "test_pr_auc": float(average_precision_score(y_test, proba_hgb)),
    "test_roc_auc": float(roc_auc_score(y_test, proba_hgb)),
    "threshold_tuned": float(thr_hgb),
    "confusion_tuned": confusion_matrix(y_test, pred_hgb).tolist(),
    "class_report_tuned": classification_report(y_test, pred_hgb, output_dict=True),
    "best_params": {k: (float(v) if hasattr(v, "item") else v) for k,v in hgb_search.best_params_.items()}
}

with open(ART / f"metrics_hgb_v1_{stamp}.json", "w") as f: 
    json.dump(metrics, f, indent=2)

with open(ART / "decision_threshold_hgb_v1.txt", "w") as f:
    f.write(str(metrics["threshold_tuned"]))

# PR/ROC plots 
prec, rec, _ = precision_recall_curve(y_test, proba_hgb)
fpr, tpr, _ = roc_curve(y_test, proba_hgb)

plt.figure(); plt.plot(rec, prec); plt.xlabel("Recall"); plt.ylabel("Precision")
plt.title(f"HGB PR curve (AP={metrics['test_pr_auc']:.3f})"); plt.grid(True, alpha=0.3)
plt.savefig(ART / f"pr_curve_hgb_v1_{stamp}.png", bbox_inches="tight"); plt.close()

plt.figure(); plt.plot(fpr, tpr); plt.plot([0,1],[0,1],'--')
plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title(f"HGB ROC curve (AUC={metrics['test_roc_auc']:.3f})")
plt.grid(True, alpha=0.3)
plt.savefig(ART / f"roc_curve_hgb_v1_{stamp}.png", bbox_inches="tight"); plt.close()

print("Saved HGB v1 artifacts:", ART)

Saved HGB v1 artifacts: /Volumes/easystore/Projects/chicago-crime-pipeline/notebooks/artifacts


In [23]:
for col in ["weekday","hour_bin","primary_type"]:
    out = slice_metrics(X_test_fe, y_test, proba_hgb, thr_hgb, col)
    if out is not None:
        out.to_csv(ART / f"slice_metrics_{col}_hgb_v1_{stamp}.csv", index=False)
        print(f"Saved slice metrics for {col}")

Saved slice metrics for weekday
Saved slice metrics for hour_bin
Saved slice metrics for primary_type


/var/folders/6z/l9wv3crd4n5bzgcrdd7vxq8m0000gn/T/ipykernel_79209/847869267.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for val, g in df.groupby(slice_col):


In [24]:
def threshold_for_recall(y_true, proba, target=0.70):
    prec, rec, thr = precision_recall_curve(y_true, proba)
    idx = np.argmax(rec >= target)
    th = thr[max(idx-1, 0)] if idx < len(thr) else 0.5
    return float(th), float(prec[max(idx-1,0)]), float(rec[max(idx-1,0)])

thr_r70, p_at_r70, r_at_r70 = threshold_for_recall(y_test, proba_hgb, target=0.70)
print("Threshold for recall≥0.70:", thr_r70, "| precision≈", p_at_r70, "| recall≈", r_at_r70)

pred_r70 = (proba_hgb >= thr_r70).astype(int)
print(classification_report(y_test, pred_r70, digits=3))
print("Confusion:\n", confusion_matrix(y_test, pred_r70))

Threshold for recall≥0.70: 0.0003309188924939405 | precision≈ 0.14401525989508823 | recall≈ 1.0
              precision    recall  f1-score   support

           0      0.000     0.000     0.000      1795
           1      0.144     1.000     0.252       302

    accuracy                          0.144      2097
   macro avg      0.072     0.500     0.126      2097
weighted avg      0.021     0.144     0.036      2097

Confusion:
 [[   0 1795]
 [   0  302]]


/Volumes/easystore/Projects/chicago-crime-pipeline/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Volumes/easystore/Projects/chicago-crime-pipeline/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Volumes/easystore/Projects/chicago-crime-pipeline/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to

In [25]:
slice_tables_r70 = {}
for col in slice_tables.keys():
    tbl = slice_metrics(X_test_fe, y_test, proba_hgb, thr_r70, col, min_support=40)
    if tbl is not None:
        slice_tables_r70[col] = tbl
        display(tbl.head(5)); display(tbl.tail(5))

,weekday,support,precision,recall,f1
0,Thursday,379,0.166227,1.0,0.285068
1,Wednesday,286,0.157343,1.0,0.271903
2,Tuesday,276,0.155797,1.0,0.269592
3,Sunday,298,0.144295,1.0,0.252199
4,Friday,311,0.138264,1.0,0.242938


,weekday,support,precision,recall,f1
2,Tuesday,276,0.155797,1.0,0.269592
3,Sunday,298,0.144295,1.0,0.252199
4,Friday,311,0.138264,1.0,0.242938
5,Monday,288,0.121528,1.0,0.216718
6,Saturday,259,0.115830,1.0,0.207612


/var/folders/6z/l9wv3crd4n5bzgcrdd7vxq8m0000gn/T/ipykernel_79209/847869267.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for val, g in df.groupby(slice_col):


,hour_bin,support,precision,recall,f1
0,18-23,622,0.159164,1.0,0.274619
1,12-17,638,0.145768,1.0,0.254446
2,00-05,411,0.138686,1.0,0.243590
3,06-11,426,0.124413,1.0,0.221294


,hour_bin,support,precision,recall,f1
0,18-23,622,0.159164,1.0,0.274619
1,12-17,638,0.145768,1.0,0.254446
2,00-05,411,0.138686,1.0,0.243590
3,06-11,426,0.124413,1.0,0.221294


,primary_type,support,precision,recall,f1
0,NARCOTICS,58,0.913793,1.0,0.954955
1,WEAPONS VIOLATION,53,0.716981,1.0,0.835165
2,CRIMINAL TRESPASS,43,0.279070,1.0,0.436364
3,OTHER OFFENSE,134,0.186567,1.0,0.314465
4,BATTERY,379,0.158311,1.0,0.273349


,primary_type,support,precision,recall,f1
7,THEFT,473,0.084567,1.0,0.155945
8,CRIMINAL DAMAGE,232,0.038793,1.0,0.074689
9,BURGLARY,96,0.031250,1.0,0.060606
10,DECEPTIVE PRACTICE,136,0.029412,1.0,0.057143
11,MOTOR VEHICLE THEFT,160,0.018750,1.0,0.036810


In [ ]:
# target mean encoding for primary_type
arrest_rate = pd.Series(y_train).groupby(X_train_fe["primary_type"]).mean()
X_train_fe["ptype_arrest_rate"] = X_train_fe["primary_type"].map(arrest_rate)
X_test_fe["ptype_arrest_rate"]  = X_test_fe["primary_type"].map(arrest_rate).fillna(arrest_rate.mean())

num_cols_fe.append("ptype_arrest_rate")

# Add interaction features to reduce time-of-day false positives\
# Combine hour_bin x primary_type
X_train_fe["ptype_x_hourbin"] = X_train_fe["primary_type"].astype(str) + "_" + X_train_fe["hour_bin"].astype(str)
X_test_fe["ptype_x_hourbin"] = X_test_fe["primary_type"].astype(str) + "_" + X_test_fe["hour_bin"].astype(str)

In [37]:
print("\n=== OVERALL (HGB v1) ===")
print("PR-AUC:", round(average_precision_score(y_test, proba_hgb), 4))
print("ROC-AUC:", round(roc_auc_score(y_test, proba_hgb), 4))
print("Tuned threshold:", float(thr_hgb))

cm = confusion_matrix(y_test, pred_hgb)
tn, fp, fn, tp = cm.ravel()
print("Confusion:", cm.tolist(), "| Precision_1:", round(tp/(tp+fp+1e-12),3),
      "| Recall_1:", round(tp/(tp+fn+1e-12),3))

print("\n=== BEST PARAMS ===")
print(hgb_search.best_params_ if 'hgb_search' in globals() else "(search object not found)")

# ----- Slice summaries (requires your 'slice_metrics' function to be defined) -----
def _safe_slice(col, k=5):
    if col not in X_test_fe.columns:
        print(f"[skip] {col} not in X_test_fe")
        return None
    tbl = slice_metrics(X_test_fe, y_test, proba_hgb, thr_hgb, col, min_support=40)
    if tbl is None or tbl.empty:
        print(f"[note] no slices (support≥40) for {col}")
        return None
    print(f"\n=== {col} — worst {k} by PRECISION (support≥40) ===")
    display(tbl.sort_values("precision").head(k)[[col,"support","precision","recall","f1"]])
    print(f"\n=== {col} — worst {k} by RECALL (support≥40) ===")
    display(tbl.sort_values("recall").head(k)[[col,"support","precision","recall","f1"]])
    return tbl

tbl_weekday   = _safe_slice("weekday", k=5)
tbl_hour_bin  = _safe_slice("hour_bin", k=5)
tbl_ptype     = _safe_slice("primary_type", k=8)

# Quick “are the new features in the model?” check
expected_new = ["ptype_arrest_rate","ptype_x_hourbin"]
print("\n=== FEATURE PRESENCE CHECK (test df) ===")
print({c: (c in X_test_fe.columns) for c in expected_new})

# Optional: print top-10 1-proba examples to inspect false positives later
idx_top_fp = np.argsort(proba_hgb)[-10:]
print("\nTop-10 predicted probabilities (for quick eyeballing):")
print(pd.DataFrame({
    "proba": proba_hgb[idx_top_fp],
    "y_true": y_test[idx_top_fp],
    "weekday": X_test_fe.iloc[idx_top_fp]["weekday"].values,
    "ptype": X_test_fe.iloc[idx_top_fp]["primary_type"].values,
    "hour_bin": X_test_fe.iloc[idx_top_fp]["hour_bin"].values
}).sort_values("proba", ascending=False).to_string(index=False))


=== OVERALL (HGB v1) ===
PR-AUC: 0.6569
ROC-AUC: 0.8878
Tuned threshold: 0.6953415078096913
Confusion: [[1698, 97], [120, 182]] | Precision_1: 0.652 | Recall_1: 0.603

=== BEST PARAMS ===
{'clf__l2_regularization': np.float64(0.003853103152262984), 'clf__learning_rate': np.float64(0.13305603957582046), 'clf__max_depth': 5, 'clf__max_iter': 227, 'clf__max_leaf_nodes': 26, 'clf__min_samples_leaf': 110}

=== weekday — worst 5 by PRECISION (support≥40) ===


,weekday,support,precision,recall,f1
6,Tuesday,276,0.490196,0.581395,0.531915
5,Wednesday,286,0.615385,0.533333,0.571429
4,Monday,288,0.625000,0.571429,0.597015
1,Friday,311,0.682927,0.651163,0.666667
2,Thursday,379,0.701754,0.634921,0.666667



=== weekday — worst 5 by RECALL (support≥40) ===


,weekday,support,precision,recall,f1
5,Wednesday,286,0.615385,0.533333,0.571429
3,Sunday,298,0.727273,0.558140,0.631579
4,Monday,288,0.625000,0.571429,0.597015
6,Tuesday,276,0.490196,0.581395,0.531915
2,Thursday,379,0.701754,0.634921,0.666667



=== hour_bin — worst 5 by PRECISION (support≥40) ===


...


=== primary_type — worst 8 by PRECISION (support≥40) ===


,primary_type,support,precision,recall,f1
11,MOTOR VEHICLE THEFT,160,0.000000,0.000000,0.000000
4,CRIMINAL TRESPASS,43,0.392857,0.916667,0.550000
10,ASSAULT,187,0.400000,0.090909,0.148148
8,THEFT,473,0.419355,0.325000,0.366197
3,OTHER OFFENSE,134,0.515152,0.680000,0.586207
7,BATTERY,379,0.607143,0.283333,0.386364
1,WEAPONS VIOLATION,53,0.730769,1.000000,0.844444
2,ROBBERY,56,0.800000,0.571429,0.666667



=== primary_type — worst 8 by RECALL (support≥40) ===


,primary_type,support,precision,recall,f1
11,MOTOR VEHICLE THEFT,160,0.000000,0.000000,0.000000
10,ASSAULT,187,0.400000,0.090909,0.148148
9,CRIMINAL DAMAGE,232,1.000000,0.111111,0.200000
6,DECEPTIVE PRACTICE,136,1.000000,0.250000,0.400000
7,BATTERY,379,0.607143,0.283333,0.386364
8,THEFT,473,0.419355,0.325000,0.366197
5,BURGLARY,96,1.000000,0.333333,0.500000
2,ROBBERY,56,0.800000,0.571429,0.666667



=== FEATURE PRESENCE CHECK (test df) ===
{'ptype_arrest_rate': True, 'ptype_x_hourbin': True}

Top-10 predicted probabilities (for quick eyeballing):
   proba  y_true   weekday     ptype hour_bin
0.994074       1    Monday NARCOTICS    00-05
0.993203       1  Thursday NARCOTICS    18-23
0.992877       1 Wednesday NARCOTICS    12-17
0.992660       1   Tuesday NARCOTICS    06-11
0.992531       1  Thursday NARCOTICS    18-23
0.992421       1  Thursday NARCOTICS    12-17
0.991894       1    Sunday NARCOTICS    06-11
0.991749       1   Tuesday NARCOTICS    18-23
0.991618       1   Tuesday NARCOTICS    18-23
0.991319       1    Friday NARCOTICS    18-23
